In [2]:
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
# 모델의 입력 차원 구성

img_rows = 28
img_cols = 28
channels = 1

# 입력 이미지 차원
img_shape = (img_rows, img_cols, channels)

# 생성자 입력으로 사용될 잡음 벡터의 크기
z_dim = 100


In [4]:
# 생성자 구현현
def build_generator(img_shape, z_dim):

  model = Sequential()
  model.add(Dense(128, input_dim = z_dim))
  model.add(LeakyReLU(alpha=0.01)) # 그냥 렐루 보단 리키렐루를 겐에서 사용하는 것이 좋다.
  model.add(Dense(28 * 28 * 1, activation = 'tanh')) # 활성화 함수는 하이퍼블릭 탄젠트 극성을 갖는게 좋다.
  # 생성자의 출력을 이미지 차원으로 변경
  model.add(Reshape(img_shape)) # 뤼 쉐입 벡타를 받아서 이미지로 리턴 해주기위함이다.

  return model

In [5]:
# 판별자 구현현

def build_discriminator(img_shape):

  model = Sequential()
  # 입력 이미지를 일렬로 펼치기
  model.add(Flatten(input_shape = img_shape))
  model.add(Dense(128))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Dense(1, activation='sigmoid')) # 덴스 레이어는 0 과 1만 구분,(참 거짓), 시그모이드 사용, 소프트 맥스도 사용 가능능

  return model

In [6]:
# 모델을 합치는 것

def build_gan(generator, discriminator):
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  return model

In [7]:
# 컴파일 하기기

discriminator = build_discriminator(img_shape)
discriminator.compile(loss = 'binary_crossentropy', optimizer = Adam(), metrics = ['accuracy'])

generator = build_generator(img_shape, z_dim)
discriminator.trainable = False # 생성자 훈련시킬 때만만 판별자의 훈련 기능을 오프프

# 생성자를 훈련하기 위해 동결된 판별자로 GAN 모델을 만들고 컴파일
gan = build_gan(generator, discriminator)
gan.compile(loss = 'binary_crossentropy', optimizer = Adam())

In [8]:
# 겐 훈련 하기 1/3

losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
  # MNIST 데이터셋 로드, [-1, 1] 범위로 노멀라이즈 수행행
  (X_train, _), (_, _) = mnist.load_data()
  X_train = X_train / 127.5 - 1.0
  X_train = np.expand_dims(X_train, axis = 3)

  # 진짜 이미지 레이블: 모두 1
  real = np.ones((batch_size, 1))

  # 가짜 이미지 레이블: 모두 0
  fake = np.zeros((batch_size, 1))

    # 위 함수에 이어서서
  for iteration in range(iterations):
    # -------------------------
    #  판별자 훈련
    # -------------------------

     # 진짜 이미지에서 랜덤 배치 가져오기
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

      # 가짜 이미지 배치 생성
    z = np.random.normal(0, 1, (batch_size, 100)) # 노말 분포로 부터 가짜이미지 배치 생성 평균 0, 표편 1
    gen_imgs = generator.predict(z)

     # 판별자 훈련
    d_loss_real = discriminator.train_on_batch(imgs, real)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)

    

    z = np.random.normal(0, 1, (batch_size, 100))
    gen_imgs = generator.predict(z)

    # 생성자 훈련
    g_loss = gan.train_on_batch(z, real)

    if (iteration + 1) % sample_interval == 0:
        # 훈련이 끝난 후 그래프를 그리기 위해 손실과 정확도 저장
        losses.append((d_loss, g_loss))
        accuracies.append(100.0 * accuracy)
        iteration_checkpoints.append(iteration + 1)

        # 훈련 과정 출력
        print("%d [D 손실: %f, 정확도: %.2f%%] [G 손실: %f]" %(iteration + 1, d_loss, 100.0 * accuracy, g_loss))

        # 생성된 이미지 샘플 출력
        sample_images(generator)

In [9]:
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):
  # 랜덤한 잡음 샘플링
  z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

  # 랜덤한 잡음에서 이미지 생성하기
  gen_imgs = generator.predict(z)

  # 이미지 픽셀 값을 [0, 1] 사이로 스케일 조정
  gen_imgs = 0.5 * gen_imgs + 0.5

  # 이미지 그리드 설정
  fig, axs = plt.subplots(image_grid_rows,image_grid_columns,figsize=(4, 4),sharey=True,sharex=True)

  cnt = 0
  for i in range(image_grid_rows):
    for j in range(image_grid_columns):
      # 이미지 그리드 출력
      axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
      axs[i, j].axis('off')
      cnt += 1

In [16]:
# 하이퍼파라미터 설정
iterations = 20000
batch_size = 128
sample_interval = 1000

# 지정된 반복 횟수만큼 GAN 훈련
train(iterations, batch_size, sample_interval)

4/4 [==============================] - 7s 6ms/step


TypeError: ignored

In [11]:
# 판별자와 생성자의 트레인 로스스
import matplotlib.pyplot as plt

losses = np.array(losses)

# 판별자와 생성자의 훈련 손실 그래프
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, losses.T[0], label="Discriminator loss")
plt.plot(iteration_checkpoints, losses.T[1], label="Generator loss")
plt.xticks(iteration_checkpoints, rotation=90)
plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()

IndexError: ignored

<Figure size 1080x360 with 0 Axes>

In [ ]:
accuracies = np.array(accuracies)
# 판별자의 정확도 그래프
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, accuracies, label="Discriminator accuracy")
plt.xticks(iteration_checkpoints, rotation=90)
plt.yticks(range(0, 100, 5))
plt.title("Discriminator Accuracy")
plt.xlabel("Iteration")
plt.ylabel("Accuracy (%)")
plt.legend()

### DCGAN

In [12]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D, Input, Dense, Reshape, Flatten, Dropout

# generator model
generator = Sequential()
generator.add(Dense(128*7*7, input_dim = 100, activation = LeakyReLU(0.2)))
# 128은 임의로 정한 노드의 수, 7*7은 이미지의 최초 크기(나중에 28*28로 올려줄예정)
generator.add(BatchNormalization()) #겐에서 배치 노멀 라이즈가 아주 중요하다.
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size = 5, padding = 'same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size = 5, padding = 'same', activation = 'tanh'))


In [13]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D, Input, Dense, Reshape, Flatten, Dropout

# discriminator model
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size = 5, strides = 2, input_shape = (28, 28, 1), padding = 'same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size = 5, strides = 2, padding = 'same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation = 'sigmoid'))
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'adam')
discriminator.trainable = False

In [14]:
# 생성자와 판별자 연결

# gan: discriminator + generator
ginput = Input(shape = (100, ))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss = 'binary_crossentropy', optimizer = 'adam')
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_3 (Sequential)   (None, 28, 28, 1)         865281    
                                                                 
 sequential_4 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [15]:
# gan training function 훈련 함수 만들기기
def gan_train(epoch, batch_size, saving_interval):
  (X_train, _), (_, _) = mnist.load_data()
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

  X_train = (X_train - 127.5)/127.5

  # 진짜이미지 만들기기
  true = np.ones((batch_size, 1))
  idx = np.random.randint(0, X_train.shape[0], batch_size)
  imgs = X_train[idx]
  d_loss_real = discriminator.train_on_batch(imgs, true)

  # 가짜이미지 만들기기
  fake = np.zeros((batch_size, 1))
  noise = np.random.normal(0, 1, (batch_size, 100))
  gen_imgs = generator.predict(noise)
  d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

  # 로스 값값
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
  g_loss = gan.train_on_batch(noise, true)

  print('epoch: {}, d_loss: {}, g_loss: {}'.format(i, d_loss, g_loss))